In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as data
import warnings
from bs4 import BeautifulSoup
import re
!python -m spacy download en_core_web_lg
import spacy
import string
import geonamescache
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
df = pd.read_csv('https://github.com/anishaga/Machine-Learning/blob/main/Assignment%201/IMDB%20Dataset.csv?raw=true')
df['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [18]:
#Removing Hyperlinks
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
df['review']=df['review'].apply(denoise_text)

In [19]:
#Removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
df['review']=df['review'].apply(remove_special_characters)

In [20]:
df['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production The filming tech...
2        I thought this was a wonderful way to spend ti...
3        Basically theres a family where a little boy J...
4        Petter Matteis Love in the Time of Money is a ...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [27]:
nlp = spacy.load('en_core_web_lg')
ner = nlp.get_pipe('ner')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [28]:
dt = pd.DataFrame(columns=["names", "Entity"])
dt.head()

,names,Entity


In [29]:
spacy.explain("GPE")
spacy.explain ("PERSON")

'People, including fictional'

In [30]:
# gets dictionary for city names
gc = geonamescache.GeonamesCache()      
cities = gc.get_cities()
cities = {v:0 for _, val in cities.items() for k, v in val.items() if k=='name'}

In [31]:
dt = pd.DataFrame()
for text in df.review.loc[:5000]:
    
    doc = nlp(text)
    for ent in doc.ents:
        
        if (ent.label_=="PERSON"):
            dt = dt.append({"names":ent.text, "Entity":"Person"}, ignore_index=True)

        if (ent.label_=="GPE"):
            if ent.text in cities:
                dt = dt.append({"names":ent.text, "Entity":"City"}, ignore_index = True) 
dt.head()


,names,Entity
0,Watching Oz,Person
1,Michael Sheen,Person
2,Williams,Person
3,Woody Allen,Person
4,Scarlet Johanson,Person


In [32]:
dt.to_json(r"..\data\ner.json", orient="records")


In [33]:
pd.read_json(r"..\data\ner.json")


,names,Entity
0,Watching Oz,Person
1,Michael Sheen,Person
2,Williams,Person
3,Woody Allen,Person
4,Scarlet Johanson,Person
...,...,...
24788,Richard Pryor,Person
24789,Michael J Pollards,Person
24790,Larry Bishops,Person
24791,Michael Madsen,Person


In [38]:
# importing packages
!pip install flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok
import pandas as pd

# intialize object
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/type/<inpt>")
def get_type(inpt):
    dt = pd.read_json(r"..\data\ner.json")
    try:
        typ = dt[dt.names==inpt].Entity[0]        
        return typ
    except:
        return "not valid input, kindly enter another input"

if __name__=="__main__":
    app.run()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f494-34-71-166-157.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 12:53:37] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 12:53:40] "GET /favicon.ico HTTP/1.1" 404 -
